# Ch.3
## 3.1 Tensorboard
*Tensorboardを利用するために必要なプロセス*  
  
1. モデル構築時に必要なところでログを取得するオペレーションを記載する。
2. 全てログを配置し終わったら、そのログをマージするオペレーションを記載する。
3. `tf.summary.FireWriter`を呼び出してログの出力先と対象グラフを指定する。
4. ログを取得するタイミングでマージするオペレーションを実行する。
5. 実行結果を`FireWriter`に追加する。
6. 出力されたログを`tensorboard`コマンドで指定して呼び出す。
7. ブラウザで対象サーバーにアクセスする。

### Images
バイナリデータ化した画像がモデルに正しく渡っているか、オーギュメンテーション後の画像が判別可能なレベルかなどを確認するのに有効。  
input imageが[mini batch size, 784]なので、[mini batch size, hight, width, channel]に変換して表示させる。  
  
### Graph
計算グラフを可視化する`Graph`は、特にログを取得するオペレーションを記載する必要がない。  
`FireWriter`を宣言する際に対象のグラフを指定してログ出力ディレクトリを指定した時点で勝手にログが作成される（後述）。  
  
### name_scope  
TensorFlowではシンプルな構成のネットワークでもオペレーションの数が多くなることが少なくない。  
しかし、そのままGraphを表示すると、全てのオペレーションが表示されてしまい可読性が下がる。  
そこで、`name_scope`を指定して各層や特定の処理ごとにまとめると、整理されてわかりやすくなる。  
### Scalars 
`loss`と`accuracy`の処理の後に`Scalars`を配置する。  
`tf.summary.scalar()`の入力値はスカラー（0階テンソル）であることに注意。  
→重みやバイアスはスカラーではないので、`Scalars`が使えない。  
→そういう場合は`tf.summary.histgram()`を利用する。  
  
### Distributions / Histogram  
引数の制約はほとんど無く、数値データのテンソルであれば何でも構わない。  
ただし、NeuralNetworkが発散した場合に`Ｎａｎ`となる場合があり、`Ｎａｎ`がひとつでもあるとエラーを吐くので注意。  
`tf.summary.histogram()`に値があれば、TensorBoardでは`Histograms`と`Distributions`の両方で結果を見ることができる。  
- Distributions:  確率分布/推移を時間軸ごとに見ることができる  
- Histogram:　ステップの度数分布を確認できる  
  
### Merging  
ログのマージには2種類あり、一部のログをマージするものと、全てのログをマージするもの。  
当たり前だが、後者は全てのログの配置が終わった後にマージすること。  

```
# merging some logs
merge = tf.summary.merge([a, b, c])

# merging all the logs
summary_op = tf.summary.merge_all()
```

### FileWriter
取得したログを実際にログファイルに書き出すのが`tf.summary.FireWriter`で、宣言するだけで計算グラフを`Graphs`に出力してくれる。  
第一引数はログ出力対象のディレクトリで、絶対パスでも相対パスでも書ける。  
  
  
### TensorBoard  
TensorBoardを起動するには、コマンドラインから以下のコマンドを実行する。  
（`path/to/log-directory`には実際にログが保存されているパスを入力）  
  
`tensorboard --logdir /path/to/log-directory`  
  
起動後は以下のURLからアクセスできる。  
http://localhost:6006

In [1]:
#!/usr/bin/env python
# coding=utf-8

#!/usr/bin/env python
# coding=utf-8

import tensorflow as tf

def mnist_tb():
    # importing MNIST data
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets('mnist_data/', one_hot=True)

    # getting train images, labeled data as batch size = 50
    train_images, train_labels = mnist.train.next_batch(50)

    # getting all the test images
    test_images = mnist.test.images

    # getting all the test labels
    test_labels = mnist.test.labels

    # input layer
    # using None for an arbitrary batch size
    # this is really useful when train and validation batches
    # are different size
    x = tf.placeholder(tf.float32, [None, 784])

    # getting a log of the input image
    # x is [None, 784] and need a transformation into an image
    # img: [size of the mini batch, hight, width, channel]
    img = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input_data', img, 10)

    # hidden layer
    with tf.name_scope('hidden'):
        w_1 = tf.Variable(tf.truncated_normal([784, 64], stddev=0.1), name='w1')
        b_1 = tf.Variable(tf.zeros([64]), name='b1')
        h_1 = tf.nn.relu(tf.matmul(x, w_1) + b_1)

        # logging
        tf.summary.histogram('w_1', w_1)

    # output layer
    with tf.name_scope('output'):
        w_2 = tf.Variable(tf.truncated_normal([64, 10], stddev=0.1), name='w2')
        b_2 = tf.Variable(tf.zeros([10]), name='b2')
        out = tf.nn.softmax(tf.matmul(h_1, w_2) + b_2)

    # loss function
    # computing mean with tf.reduce_mean
    y = tf.placeholder(tf.float32, [None, 10])
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.square(y - out))
        # logging
        tf.summary.scalar('loss', loss)

    # train
    with tf.name_scope('train'):
        train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # evaluation
    with tf.name_scope('accuracy'):
        correct = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        # logging
        tf.summary.scalar('accuracy', accuracy)

    # merging all the logs
    summary_op = tf.summary.merge_all()

    # defining initializer
    init = tf.global_variables_initializer()

    with tf.Session() as sess:

        sess.run(init)

        # FileWriter
        summary_writer = tf.summary.FileWriter('logs', sess.graph)

        # loading test data
        test_images = mnist.test.images
        test_labels = mnist.test.labels

        for i in range(1000):
            step = i + 1
            train_images, train_labels = mnist.train.next_batch(50)
            sess.run(train_step, feed_dict={x:train_images, y:train_labels})

            if step % 10 == 0:
                # logging (outputs are protocol buffers of logs)
                summary_str = sess.run(summary_op,
                                       feed_dict={x:test_images, y:test_labels})
                # writing protocol buffers of logs
                summary_writer.add_summary(summary_str, step)

                # computing and printing accuracy
                acc_val = sess.run(accuracy,
                                   feed_dict={x:test_images, y:test_labels})
                print('Step {}: accuracy = {}'.format(step, acc_val))

/home/tk/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist_tb()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Step 10: accuracy = 0.14470000565052032
Step 20: accuracy = 0.18539999425411224
Step 30: accuracy = 0.23000000417232513
Step 40: accuracy = 0.2732999920845032
Step 50: accuracy = 0.3125999867916107
Step 60: accuracy = 0.3368000090122223
Step 70: accuracy = 0.3635999858379364
Step 80: accuracy = 0.399